In [1]:
#Check Python Version
import shutil
import sys
import os
sys.version


#Upgrade pip
%pip install --upgrade pip


#Check TF & TFX Versioning
import tensorflow as tf
print(tf.__version__)
from tfx import v1 as tfx
print(tfx.__version__)


# Pipeline name
PIPELINE_NAME = "copy_tfrecords"

# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = './artifacts'
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# Folder path to data
DATA_ROOT = './data/'

# Folder path to tfrecords
TFRECORDS_TRAIN_DATA_PATH = '../../../../Documents/tfrecord1_split_train/data_tfrecord-00000-of-00001.gz'
TFRECORDS_EVAL_DATA_PATH = '../../../../Documents/tfrecord2_split_eval/data_tfrecord-00000-of-00001.gz'

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

2.11.0


1.12.0


In [11]:
import json
import tempfile
from typing import Any, Dict, List
from tfx.v1.types.standard_artifacts import Examples
from tfx.types import artifact_utils
from typing import Any, Dict, List, Optional, Type, Union


from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

from hello import component

from tfx.dsl.component.experimental.decorators import component

from tfx.dsl.io import fileio

@component
def CopyExampleGen(
        input_dict: tfx.dsl.components.Parameter[Dict[str, str]],
        output_example: tfx.dsl.components.OutputArtifact[Examples]
      ) -> tfx.dsl.components.OutputDict():
    
        """TODO: Function that parses input_dict, creates a directory from the split-names and tfrecord uris provided"""

        input_train_dir='./examples_imported/Split-train/data_tfrecord-00000-of-00001.gz'
        input_eval_dir='examples_imported/Split-eval/data_tfrecord-00000-of-00001.gz'
        split_names='["train","eval"]'

        output_example.split_names=artifact_utils.encode_split_names(['train', 'eval'])

        fileio.mkdir(output_example.uri+'/Split-train/')
        fileio.mkdir(output_example.uri+'/Split-eval/')

        fileio.copy(input_train_dir, output_example.uri+'/Split-train/data_tfrecord-00000-of-00001.gz', True) 
        fileio.copy(input_eval_dir, output_example.uri+'/Split-eval/data_tfrecord-00000-of-00001.gz', True) 


def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  # User input dictionary with split-names and their resepctive uri to tfrecords
  tfrecords_dict: Dict[str, str]={
    "train":TFRECORDS_TRAIN_DATA_PATH,
    "eval":TFRECORDS_EVAL_DATA_PATH
  }
  split_names=''
  counter=1

#   cwd = os.getcwd()
#   temp_dir = tempfile.TemporaryDirectory(dir = cwd)

#   print(temp_dir.name)

  # Destination directory for source
  destination_examples_artifact_uri = './examples'
  print(destination_examples_artifact_uri)

  if not os.path.exists(destination_examples_artifact_uri):
      os.mkdir(destination_examples_artifact_uri)


  # Create a folder in destination_examples_artifact_uri for every key name
  for key in tfrecords_dict:
        split_name_temp = "Split-"+key
        path = os.path.join(destination_examples_artifact_uri, split_name_temp)
        if not os.path.exists(path):
            os.mkdir(path)
            print(path)
        shutil.copy(tfrecords_dict[key], path)
        print(path)


  # Create split_names string
  for key in tfrecords_dict:
      if(counter==1):
          split_names+='["'+key+'","'
          counter=counter+1
      elif(counter==2):
          split_names+=key+'"]'
      else:
          split_names+=key+','
          counter=counter+1

  print(split_names)

  # CopyExampleGen execution process to replace Importer Node functionality
  examples_importer = tfx.dsl.Importer(
        source_uri=destination_examples_artifact_uri,
        artifact_type=tfx.types.standard_artifacts.Examples,
        # properties={"split_names": '["train","eval"]'}
        properties={"split_names": split_names}
        ).with_id('examples_importer')


#   output_example: tfx.dsl.components.OutputArtifact[tfx.types.standard_artifacts.Examples]
                                                    
  copy_examples=CopyExampleGen(
      input_dict=tfrecords_dict,
    #   output_example=output_example
  )

  
#   hello_component = component.HelloComponent(
#       input_data=examples_importer.outputs['result'],
#       tfrecords_dict=tfrecords_dict
#     #   input_data=tfrecords_dict
#   )

  # Test downstream component
  statistics_gen = tfx.components.StatisticsGen(
     examples=copy_examples.outputs['output_example'])

  # Following three components will be included in the pipeline.
  components = [
      # example_gen,
      copy_examples,
      examples_importer,
    #   hello_component,
      statistics_gen
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

In [12]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      metadata_path=METADATA_PATH)
  )

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Using deployment config:
 executor_specs {
  key: "CopyExampleGen"
  value {
    python_class_executable_spec {
      class_path: "__main__.CopyExampleGen_Executor"
    }
  }
}
executor_specs {
  key: "StatisticsGen"
  value {
    beam_executable_spec {
      python_executor_spec {
        class_path: "tfx.components.statistics_gen.executor.Executor"
      }
    }
  }
}
metadata_connection_config {
  database_connection_config {
    sqlite {
      filename_uri: "metadata/copy_tfrecords/metadata.db"
      connection_mode: READWRITE_OPENCREATE
    }
  }
}

INFO:absl:Using connection config:
 sqlite {
  filename_uri: "metadata/copy_tfrecords/metadata.db"
  connection_mode: READWRITE_OPENCREATE
}

INFO:absl:Component CopyExampleGen is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "__main__.CopyExampleGen"
  }
  id: "CopyExampleGen"
}
contexts {
  contexts {
    type {
      name: "pipeline"


./examples
./examples/Split-train
./examples/Split-eval
["train","eval"]


INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Component CopyExampleGen is finished.
INFO:absl:Component examples_importer is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "tfx.dsl.components.common.importer.Importer"
  }
  id: "examples_importer"
}
contexts {
  contexts {
    type {
      name: "pipeline"
    }
    name {
      field_value {
        string_value: "copy_tfrecords"
      }
    }
  }
  contexts {
    type {
      name: "pipeline_run"
    }
    name {
      field_value {
        string_value: "2023-03-13T19:14:17.207524"
      }
    }
  }
  contexts {
    type {
      name: "node"
    }
    name {
      field_value {
        string_value: "copy_tfrecords.examples_importer"
      }
    }
  }
}
outputs {
  outputs {
    key: "result"
    value {
      artifact_spec {
        type {
          name: "Examples"
          properties {
            key: "span"
            value: INT
          }
          properties {
            key: "sp

In [8]:
from typing import Dict
from tfx.v1.types.standard_artifacts import Examples
from tfx.types import artifact_utils
from typing import Dict

from tfx.dsl.component.experimental.decorators import component

from tfx.dsl.io import fileio

@component
def CopyExampleGen(
        input_dict: tfx.dsl.components.Parameter[Dict[str, str]],
        output_example: tfx.dsl.components.OutputArtifact[Examples]
      ) -> tfx.dsl.components.OutputDict():
    
        """TODO: Function that parses input_dict, creates a directory from the split-names and tfrecord uris provided"""

        input_train_dir='./examples_imported/Split-train/data_tfrecord-00000-of-00001.gz'
        input_eval_dir='examples_imported/Split-eval/data_tfrecord-00000-of-00001.gz'
        split_names='["train","eval"]'

        output_example.split_names=artifact_utils.encode_split_names(['train', 'eval'])

        fileio.mkdir(output_example.uri+'/Split-train/')
        fileio.mkdir(output_example.uri+'/Split-eval/')

        fileio.copy(input_train_dir, output_example.uri+'/Split-train/data_tfrecord-00000-of-00001.gz', True) 
        fileio.copy(input_eval_dir, output_example.uri+'/Split-eval/data_tfrecord-00000-of-00001.gz', True) 



In [10]:
from typing import Dict
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

"""Prepare variables"""
context=InteractiveContext()
tfrecords_dict: Dict[str, str]={
    "train":TFRECORDS_TRAIN_DATA_PATH,
    "eval":TFRECORDS_EVAL_DATA_PATH
  }


"""Call CopyExampleGen Component"""
copy_examples=CopyExampleGen(input_dict=tfrecords_dict)

context.run(copy_examples)


"""Call StatisticsGen Component"""
statistics_gen=tfx.components.StatisticsGen(examples=copy_examples.outputs['output_example'])

context.run(statistics_gen)



INFO:absl:Running driver for CopyExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for CopyExampleGen
INFO:absl:Running publisher for CopyExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to /tmp/tfx-interactive-2023-03-13T19_13_19.034821-4xshqe2g/StatisticsGen/statistics/2/Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to /tmp/tfx-interactive-2023-03-13T19_13_19.034821-4xshqe2g/StatisticsGen/statistics/2/Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))